In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-4b-it")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

W0923 00:47:30.683000 48078 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [3]:
print(model)

Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwi

### Data Prep

In [5]:
from datasets import load_dataset
dataset = load_dataset("AI4Math/MathVista",split="testmini")

README.md: 0.00B [00:00, ?B/s]

data/testmini-00000-of-00001-725687bf7a1(…):   0%|          | 0.00/142M [00:00<?, ?B/s]

data/test-00000-of-00002-6b81bd7f7e2065e(…):   0%|          | 0.00/358M [00:00<?, ?B/s]

data/test-00001-of-00002-6a611c71596db30(…):   0%|          | 0.00/386M [00:00<?, ?B/s]

Generating testmini split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5141 [00:00<?, ? examples/s]

In [16]:
def is_numeric_answer(example):
  try:
    float(example["answer"])
    return True
  except:
    return False

dataset = dataset.filter(is_numeric_answer)
print(len(dataset))

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

566


In [17]:
def resize_images(example):
    image = example["decoded_image"]
    image = image.resize((512,512))
    example["decoded_image"] = image
    return example

dataset = dataset.map(resize_images)
def convert_to_rgb(example):
    image = example["decoded_image"]
    if image.mode != "RGB":
        image = image.convert("RGB")
    example["decoded_image"] = image
    return example

dataset = dataset.map(convert_to_rgb)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

In [18]:
# Define the delimiter variables for clarity and easy modification
REASONING_START = "<REASONING>"
REASONING_END = "</REASONING>"
SOLUTION_START = "<SOLUTION>"
SOLUTION_END = "</SOLUTION>"

def make_conversation(example):
    # Define placeholder constants if they are not defined globally

    # The user's text prompt
    text_content = (
        f"{example['question']}, provide your reasoning between {REASONING_START} and {REASONING_END} "
        f"and then your final answer between {SOLUTION_START} and (put a float here) {SOLUTION_END}"
    )

    # Construct the prompt in the desired multi-modal format
    prompt = [
        {
            "role": "user",
            "content": [
                {"type": "image"},  # Placeholder for the image
                {"type": "text", "text": text_content},  # The text part of the prompt
            ],
        },
    ]

    # The actual image data is kept separate for the processor
    return {"prompt": prompt, "image": example["decoded_image"], "answer": example["answer"]}

train_dataset = dataset.map(make_conversation)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

In [20]:
train_dataset = train_dataset.remove_columns("image")

In [21]:
train_dataset = train_dataset.rename_column("decoded_image", "image")

In [22]:
train_dataset[0]

{'pid': '1',
 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?",
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512>,
 'choices': None,
 'unit': None,
 'precision': 1.0,
 'answer': '1.2',
 'question_type': 'free_form

In [23]:
train_dataset = train_dataset.map(
    lambda example: {
        "prompt": tokenizer.apply_chat_template(
            example["prompt"],
            tokenize=False,
            add_generation_prompt=False
        )
    }
)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

In [24]:
train_dataset[0]

{'pid': '1',
 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?",
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512>,
 'choices': None,
 'unit': None,
 'precision': 1.0,
 'answer': '1.2',
 'question_type': 'free_form

In [25]:
# copy-unsloth
# Reward functions
def formatting_reward_func(completions,**kwargs):
    import re
    thinking_pattern = f'{REASONING_START}(.*?){REASONING_END}'
    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    scores=[]
    for completion in completions :
      score=0
      thinking_matches = re.findall(thinking_pattern, completion, re.DOTALL)
      answer_matches = re.findall(answer_pattern, completion, re.DOTALL)
      if len(thinking_matches) == 1 :   # if there are thinking
        score +=1.0
      if len(answer_matches) == 1 :    # if there are answer
        score +=1.0
      scores.append(score)
    return scores


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    import re   # if answer correctly

    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    responses = [re.findall(answer_pattern, completion, re.DOTALL) for completion in completions]
    q = prompts[0]

    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:{completions[0]}")
    return [2.0 if len(r)==1 and a == r[0].replace('\n','') else 0.0 for r, a in zip(responses, answer)]


In [26]:
train_dataset[0]["prompt"]

"<bos><start_of_turn>user\n<start_of_image>When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?, provide your reasoning between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a float here) </SOLUTION><end_of_turn>\n"

### Model

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [1]:
from dawnet.experimentals.qwen3.model import Qwen3Model, Qwen3Tokenizer, generate

In [2]:
cfg = Qwen3Model.default_config("4B")

In [29]:
cfg

{'vocab_size': 151936,
 'context_length': 40960,
 'emb_dim': 2560,
 'n_heads': 32,
 'n_layers': 36,
 'hidden_dim': 9728,
 'head_dim': 128,
 'qk_norm': True,
 'n_kv_groups': 8,
 'rope_base': 1000000.0,
 'dtype': 'bf16'}

In [3]:
model = Qwen3Model(cfg)

In [4]:
prefix = "/Users/john/.cache/huggingface/hub/models--Qwen--Qwen3-4B/snapshots/1cfa9a7208912126459214e8b04321603b3df60c"

In [5]:
import json
from pathlib import Path

In [6]:
with Path(prefix, "model.safetensors.index.json").open("r") as fi:
    data = json.load(fi)

In [36]:
cfg

{'vocab_size': 151936,
 'context_length': 40960,
 'emb_dim': 2560,
 'n_heads': 32,
 'n_layers': 36,
 'hidden_dim': 9728,
 'head_dim': 128,
 'qk_norm': True,
 'n_kv_groups': 8,
 'rope_base': 1000000.0,
 'dtype': torch.bfloat16}

In [ ]:
{
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.0",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}


In [35]:
data

{'metadata': {'total_size': 8044936192},
 'weight_map': {'model.embed_tokens.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.input_layernorm.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.mlp.down_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.mlp.gate_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.mlp.up_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.post_attention_layernorm.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.k_norm.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.k_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.o_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.q_norm.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.q_proj.weight': 'model-00001-of-00003.safetensors',
  'model.layers.0.self_attn.v_proj.weight': 'model-00001-of-00003.safetensors',
  

In [7]:
import safetensors

In [8]:
st1 = safetensors.torch.load_file(Path(prefix, "model-00001-of-00003.safetensors"))

In [9]:
st2 = safetensors.torch.load_file(Path(prefix, "model-00002-of-00003.safetensors"))
st3 = safetensors.torch.load_file(Path(prefix, "model-00003-of-00003.safetensors"))

In [54]:
st3.keys()

dict_keys(['model.layers.35.input_layernorm.weight', 'model.layers.35.mlp.down_proj.weight', 'model.layers.35.mlp.up_proj.weight', 'model.layers.35.post_attention_layernorm.weight', 'model.norm.weight'])

In [57]:
model.layers[9].mlp.down_proj

AttributeError: 'Qwen3Model' object has no attribute 'layers'

In [58]:
model.trf_blocks[0]

TransformerBlock(
  (att): GroupedQueryAttention(
    (W_query): Linear(in_features=2560, out_features=4096, bias=False)
    (W_key): Linear(in_features=2560, out_features=1024, bias=False)
    (W_value): Linear(in_features=2560, out_features=1024, bias=False)
    (out_proj): Linear(in_features=4096, out_features=2560, bias=False)
    (q_norm): RMSNorm()
    (k_norm): RMSNorm()
  )
  (ff): FeedForward(
    (fc1): Linear(in_features=2560, out_features=9728, bias=False)
    (fc2): Linear(in_features=2560, out_features=9728, bias=False)
    (fc3): Linear(in_features=9728, out_features=2560, bias=False)
  )
  (norm1): RMSNorm()
  (norm2): RMSNorm()
)

In [10]:
st1.update(st2)
st1.update(st3)

In [11]:
model.load_weights(st1)

Model uses weight tying.


In [70]:
model.to(torch.device("mps"))

Qwen3Model(
  (tok_emb): Embedding(151936, 2560)
  (trf_blocks): ModuleList(
    (0-35): 36 x TransformerBlock(
      (att): GroupedQueryAttention(
        (W_query): Linear(in_features=2560, out_features=4096, bias=False)
        (W_key): Linear(in_features=2560, out_features=1024, bias=False)
        (W_value): Linear(in_features=2560, out_features=1024, bias=False)
        (out_proj): Linear(in_features=4096, out_features=2560, bias=False)
        (q_norm): RMSNorm()
        (k_norm): RMSNorm()
      )
      (ff): FeedForward(
        (fc1): Linear(in_features=2560, out_features=9728, bias=False)
        (fc2): Linear(in_features=2560, out_features=9728, bias=False)
        (fc3): Linear(in_features=9728, out_features=2560, bias=False)
      )
      (norm1): RMSNorm()
      (norm2): RMSNorm()
    )
  )
  (final_norm): RMSNorm()
  (out_head): Linear(in_features=2560, out_features=151936, bias=False)
)

In [12]:
print(model)

Qwen3Model(
  (tok_emb): Embedding(151936, 2560)
  (trf_blocks): ModuleList(
    (0-35): 36 x TransformerBlock(
      (att): GroupedQueryAttention(
        (W_query): Linear(in_features=2560, out_features=4096, bias=False)
        (W_key): Linear(in_features=2560, out_features=1024, bias=False)
        (W_value): Linear(in_features=2560, out_features=1024, bias=False)
        (out_proj): Linear(in_features=4096, out_features=2560, bias=False)
        (q_norm): RMSNorm()
        (k_norm): RMSNorm()
      )
      (ff): FeedForward(
        (fc1): Linear(in_features=2560, out_features=9728, bias=False)
        (fc2): Linear(in_features=2560, out_features=9728, bias=False)
        (fc3): Linear(in_features=9728, out_features=2560, bias=False)
      )
      (norm1): RMSNorm()
      (norm2): RMSNorm()
    )
  )
  (final_norm): RMSNorm()
  (out_head): Linear(in_features=2560, out_features=151936, bias=False)
)


In [15]:
from peft import LoraConfig, TaskType, get_peft_model

In [17]:
peft_model = get_peft_model(model, LoraConfig(peft_type=TaskType.CAUSAL_LM, r=16, lora_alpha=32, target_modules=["fc1"]))

In [18]:
print(peft_model)

PeftModel(
  (base_model): LoraModel(
    (model): Qwen3Model(
      (tok_emb): Embedding(151936, 2560)
      (trf_blocks): ModuleList(
        (0-35): 36 x TransformerBlock(
          (att): GroupedQueryAttention(
            (W_query): Linear(in_features=2560, out_features=4096, bias=False)
            (W_key): Linear(in_features=2560, out_features=1024, bias=False)
            (W_value): Linear(in_features=2560, out_features=1024, bias=False)
            (out_proj): Linear(in_features=4096, out_features=2560, bias=False)
            (q_norm): RMSNorm()
            (k_norm): RMSNorm()
          )
          (ff): FeedForward(
            (fc1): lora.Linear(
              (base_layer): Linear(in_features=2560, out_features=9728, bias=False)
              (lora_dropout): ModuleDict(
                (default): Identity()
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=2560, out_features=16, bias=False)
              )
              (lora_

In [26]:
for name, param in model.named_parameters():
    print(name, param.requires_grad) 

tok_emb.weight False
trf_blocks.0.att.W_query.weight False
trf_blocks.0.att.W_key.weight False
trf_blocks.0.att.W_value.weight False
trf_blocks.0.att.out_proj.weight False
trf_blocks.0.att.q_norm.scale False
trf_blocks.0.att.k_norm.scale False
trf_blocks.0.ff.fc1.base_layer.weight False
trf_blocks.0.ff.fc1.lora_A.default.weight True
trf_blocks.0.ff.fc1.lora_B.default.weight True
trf_blocks.0.ff.fc2.weight False
trf_blocks.0.ff.fc3.weight False
trf_blocks.0.norm1.scale False
trf_blocks.0.norm2.scale False
trf_blocks.1.att.W_query.weight False
trf_blocks.1.att.W_key.weight False
trf_blocks.1.att.W_value.weight False
trf_blocks.1.att.out_proj.weight False
trf_blocks.1.att.q_norm.scale False
trf_blocks.1.att.k_norm.scale False
trf_blocks.1.ff.fc1.base_layer.weight False
trf_blocks.1.ff.fc1.lora_A.default.weight True
trf_blocks.1.ff.fc1.lora_B.default.weight True
trf_blocks.1.ff.fc2.weight False
trf_blocks.1.ff.fc3.weight False
trf_blocks.1.norm1.scale False
trf_blocks.1.norm2.scale False
t

In [74]:
tokenizer = Qwen3Tokenizer(
tokenizer_file_path="/Users/john/Downloads/tokenizer.json",
repo_id=None,
add_generation_prompt=False,
add_thinking=False
)


In [75]:
device = torch.device("mps")

In [76]:
prompt = "Give me a short introduction to large language models."
input_token_ids = tokenizer.encode(prompt)
output = model(torch.tensor(input_token_ids, device=device).unsqueeze(0))

In [ ]:
output_token_ids = generate(
    model=model,
    idx=torch.tensor(input_token_ids, device=device).unsqueeze(0),
    max_new_tokens=512,
    context_size=40960,
    top_k=1,
    temperature=0.
)
output_text = tokenizer.decode(output_token_ids.squeeze(0).tolist())


In [ ]:
print(output_text)